In [36]:
import pandas as pd
import numpy as np
import datetime
import warnings
import time
from time import perf_counter
import glob
import tweets_utils
import csv
import importlib
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
%autosave 3600

Autosaving every 3600 seconds


- [Original tweets](#original)
    - [Mentions](#original_mention)
    
    
- [Replies](#replies)
    - [Fake reply (other mentions)](#reply_mention)
    - [Reply to status (original, reply or quote)](#reply_status)
    
    
- [Retweets](#retweets)
    - [Retweet of original](#rt_original)
    - [Retweet in repliy](#rt_reply)
    - [Retweet of  mention](#rt_mention)
    - [Retweet of quote](#rt_quote)
    
    
- [Quotes](#quotes)
    - [Quote of status (original or quote)](#quote_status)
    - [Quote in reply](#quote_reply)
    
    
- [Split check](#split_check)

In [8]:
cols = ["id", "id_str", "user_id", "user_screen_name", "text","in_reply_to_status_id",
        "in_reply_to_status_id_str" ,"in_reply_to_user_id" ,"in_reply_to_user_id_str" ,"in_reply_to_screen_name",
        "rt_created_at", "rt_in_reply_to_status_id", "rt_in_reply_to_user_id", "rt_id", "rt_user_id", "rt_created_at",
        "rt_user_screen_name", "quoted_status_id" ,"quoted_status_text" ,"quoted_status_created_at", "quoted_user_id",
        "quoted_user_created_at", "hashtags", "user_mentions", "urls"]

In [9]:
path = glob.glob("C:/Users/gianl/Desktop/Gi/Supsi/Laura/data/")[0]
df = pd.read_csv(path + "tweets.csv", lineterminator="\n", low_memory=False, usecols=cols)

# Original
<a id="original"/>

In [164]:
original = df[df["in_reply_to_screen_name"].isna() & df["rt_created_at"].isna() & df["quoted_status_id"].isna()]
# original.info()

## Original - Mentions
<a id="original_mention"/>

In [118]:
nan_value = float("NaN")
original_w_mentions = original.replace("[]", nan_value)
original_w_mentions = original_w_mentions[original_w_mentions["user_mentions"].notna()]
# original_w_mentions.info()
# original_w_mentions

# Replies
<a id="replies"/>

In [141]:
reply = df[df["in_reply_to_user_id"].notna() & df["quoted_status_id"].isna()]
# reply.info()

## Fake reply: Mention
<a id="reply_mention"/>

In [142]:
# menzione (@username) ad inizio tweet (sospeso se df["quoted_status_id"].notna())
mentions = reply[reply["in_reply_to_status_id"].isna()]
# mentions.info()
# mentions

## Reply to status
<a id="reply_status"/>

In [143]:
# risposte:
# 1281589414838362113 --> risposta A UN POST ORIGINALE, non individuabile
# 1281586483544170496 --> risposta A UNA QUOTE, non individuabile
# 1394609476263718913 --> risposta A UNA RISPOSTA, non individuabile
reply_to_status = reply[reply["in_reply_to_status_id"].notna()]
# reply_to_status.info()
# reply_to_status

### Check rp

In [125]:
len(mentions) + len(reply_to_status) == len(reply)

True

# Retweets
<a id="retweets"/>

In [101]:
retweet = df[df["rt_created_at"].notna()]
# retweet.info(verbose=True, null_counts=True)
# retweet

## Retweet of original
<a id="rt_original"/>

In [98]:
retweet_original = retweet[retweet["rt_in_reply_to_user_id"].isna() & retweet["quoted_status_id"].isna()]
# retweet_original.info(verbose=True, null_counts=True)
# retweet_original

## Retweet of reply
<a id="rt_reply"/>

In [99]:
retweet_in_reply = retweet[retweet["rt_in_reply_to_status_id"].notna()]
# retweet_in_reply.info()
# retweet_in_reply

## Retweet of mention
<a id="rt_mention"/>

In [163]:
retweet_of_mentions = retweet[retweet["rt_in_reply_to_status_id"].isna() & retweet["rt_in_reply_to_user_id"].notna()]
# retweet_of_mentions.info()
# retweet_of_mentions

## Retweet of quote
<a id="rt_quote"/>

In [152]:
retweet_quote = retweet[retweet["quoted_status_id"].notna() & retweet["rt_in_reply_to_user_id"].isna()]
# retweet_quote.info()
# retweet_quote

### Check rt

In [105]:
len(retweet_quote) + len(retweet_of_mentions) + len(retweet_in_reply) + len(retweet_original) == len(retweet)

True

# Quotes
<a id="quotes"/>

In [162]:
quote = df[df["quoted_status_id"].notna() & df["rt_created_at"].isna()]
# quote.info()

## Quotes of status
<a id="quote_status"/>

In [146]:
#  quote["quoted_user_created_at"].isna() per le quotes dei post rimossi
quote_original = quote[quote["in_reply_to_screen_name"].isna()]
# quote_original.info()
# quote_original

## Quotes in reply
<a id="quote_reply"/>

In [155]:
quote_reply = quote[quote["in_reply_to_screen_name"].notna() & quote["in_reply_to_status_id_str"].notna()]
# quote_reply.info()
# quote_reply

### Check quote

In [151]:
len(quote) == len(quote_original) + len(quote_reply)

True

# Split check
<a id="split_check"/>

In [145]:
len(df) == len(reply) + len(retweet) + len(original) + len(quote)

True